### Определение инструмента

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain_openai import ChatOpenAI
import os

In [ ]:
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")

llm = ChatOpenAI(
    model="google/gemini-2.5-flash",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

#### Базовый способ

In [ ]:
@tool
def square_root(x: float) -> float:
    """Вычислите квадратный корень из числа"""
    return x ** 0.5

##### Явное задание имени

In [ ]:
@tool("square_root")
def tool1(x: float) -> float:
    """Вычислите квадратный корень из числа"""
    return x ** 0.5

##### Явное задание имени + описания

- Теперь и имя, и описание заданы явно.
- Это лучшая практика в production: так ты контролируешь, что видит LLM.
- Особенно важно, если docstring длинный или содержит технические детали, не нужные модели.

In [ ]:
@tool("square_root", description="Вычислите квадратный корень из числа")
def tool1(x: float) -> float:
    return x ** 0.5

##### Прямой вызов инструмента
- Инструменты в LangChain поддерживают единый интерфейс invoke.
- Вход — словарь с аргументами (даже если аргумент один!).
- Выход — результат функции (21.61018...).

In [ ]:
tool1.invoke({"x": 467})

### Добавляем tool в агента

In [ ]:
agent = create_agent(
    model=llm,
    tools=[tool1],
    system_prompt="Вы — волшебник арифметики. Используйте свои инструменты, чтобы вычислить квадратный корень и квадрат любого числа."
)

In [ ]:
question = HumanMessage(content="Чему равен квадратный корень из 467?")

response = agent.invoke(
    {"messages": [question]}
)
print(response['messages'][-1].content)

In [ ]:
from pprint import pprint

pprint(response['messages'])

In [ ]:
print(response["messages"][1].tool_calls)